In [ ]:
import sys
sys.path.append(r'F:\gitlab\mxdbg\src')
from mxESP32Debugger.debugger import Dbg as MXDBG

import time
import ctypes

import toml
from tqdm import trange
from loguru import logger

# Init MXDBG

In [ ]:
dev = MXDBG()
dev.power_init()
dev.power_control(communication_type="I2C", power_type="3V3")

ret, data = dev.i2c_config(sda_pin=10, scl_pin=11, freq=400000, sda_pullup=True, scl_pullup=True)
print(ret, data)

# Read configuration files

In [ ]:
with open("adxl345.toml", "r") as f:
    config = toml.load(f)
    
reg_list = config['register_list']
slave_addr = config['address']

# Init Sensor

In [ ]:
ret, data = dev.i2c_write_read(port=0, slave_id=slave_addr, write_list=[0x00], read_length=1)
assert data[0] == 0xE5, f"Expected 0xE5, got {data[0]:#04x}"

In [ ]:
for reg in reg_list:
    ret, data = dev.i2c_write_read(port=0, slave_id=slave_addr, write_list=reg, read_length=0)
    assert ret == True, f"Failed to write register {reg:#04x}"

# Read gravity data

In [ ]:
def get_gravity_data(slave_addr):
    ret, data = dev.i2c_write_read(port=0, slave_id=slave_addr, write_list=[0x32], read_length=6)
    if ret != True:
        print("Error reading data")
        return None

    data = bytearray(data)

    x = ctypes.c_int16.from_buffer_copy(data[0:2]).value * 3.9
    y = ctypes.c_int16.from_buffer_copy(data[2:4]).value * 3.9
    z = ctypes.c_int16.from_buffer_copy(data[4:6]).value * 3.9

    return x, y, z

In [ ]:
while True:
    x, y, z = get_gravity_data(slave_addr=slave_addr)
    print("{:.2f}\t{:.2f}\t{:.2f}\r".format(x, y, z), end="")